In [1]:
!pip3 install thirdai --upgrade
!pip3 install thirdai[neural_db]
!pip3 install langchain --upgrade
!pip3 install openai --upgrade
!pip3 install paper-qa --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.4/6.4 MB 21.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.1/95.1 kB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 58.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 65.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 60.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 93.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.9/235.9 kB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.0/41.0 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 837.8/837.8 kB 55.3 MB/s eta 0:00:0

In [2]:
from thirdai import licensing, neural_db as ndb
licensing.deactivate()
licensing.activate("1FB7DD-CAC3EC-832A67-84208D-C4E39E-V3")

In [3]:
import nltk
nltk.download('punkt')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [4]:
db = ndb.NeuralDB(user_id="my_user")

In [5]:
valid_url_data = ndb.parsing_utils.recursive_url_scrape(base_url="https://www.vox.com/the-goods/23458837/sam-bankman-fried-ftx-sbf-downfall-explained", max_crawl_depth=12)

crawl depth: 12
Adding https://www.vox.com/the-goods/23458837/sam-bankman-fried-ftx-sbf-downfall-explained
Adding https://www.vox.com/the-goods/23458837/sam-bankman-fried-ftx-sbf-downfall-explained#content
Total links found: 2


In [6]:
insertable_docs = []

for url, response in valid_url_data:
    try:
        insertable_docs.append(ndb.URL(url, response))
    except:
        continue

In [7]:
insertable_docs

In [8]:
source_ids = db.insert(insertable_docs, train=False)

In [22]:
search_results = db.search(
    query="who is Sam bankman fried",
    top_k=2,
    on_error=lambda error_msg: print(f"Error! {error_msg}"))

for result in search_results:
    print(result.text)
    # print('///////')
    # print(result.context(radius=1))
    # print('\\\\\\\\')
    print(result.source)
    # print(result.metadata)
    # print('************')


In a Twitter DM interview with Future Perfect reporter Kelsey Piper following the implosion of FTX, Bankman-Fried revealed a cynical view of ethics that seemed to contradict the more nuanced views of right and wrong he'd discussed in the press before. "[M]an a lot of the dumb shit I said," he wrote. "[I]t's not true, not really."
https://www.vox.com/the-goods/23458837/sam-bankman-fried-ftx-sbf-downfall-explained#content
is reportedly missing. The stunning contrast between image and reality has resulted in Bankman-Fried facing a reputational fall from grace swifter than any in recent memory. The Justice Department and SEC began investigating FTX immediately after its collapse, and his friends and admirers in crypto, philanthropic, and political circles have quickly begun distancing themselves from the man widely dubbed the king of crypto.
https://www.vox.com/the-goods/23458837/sam-bankman-fried-ftx-sbf-downfall-explained#content


In [36]:
import os
os.environ["OPENAI_API_KEY"] = "openaikey"

In [37]:
from langchain.chat_models import ChatOpenAI
from paperqa.prompts import qa_prompt
from paperqa.chains import make_chain

llm = ChatOpenAI(
    model_name='gpt-3.5-turbo',
    temperature=0.1,
)

qa_chain = make_chain(prompt=qa_prompt, llm=llm)

In [38]:
def get_references(query):
    search_results = db.search(query,top_k=6)
    references = []
    for result in search_results:
        references.append(result.text)
        references.append(result.source)
    return references

def get_answer(query, references):
    return qa_chain.run(question=query, context='\n\n'.join(references[:3]), answer_length="abt 50 words")

In [39]:
query = "Who is sam bankman fried"

references = get_references(query)
print(references)

["In December, Bankman-Fried was arrested in the Bahamas and charged with wire fraud, securities fraud, and money laundering, among other things; he has since been extradited to the US and released from jail on a $250 million bond -- according to Reuters, the largest-ever pretrial bond. A trial date has not been set, but it is expected to take place in the Southern District of New York. Caroline Ellison and Gary Wang, two former top executives at Bankman-Fried's companies, have pleaded guilty to", 'https://www.vox.com/the-goods/23458837/sam-bankman-fried-ftx-sbf-downfall-explained#content', 'In a Twitter DM interview with Future Perfect reporter Kelsey Piper following the implosion of FTX, Bankman-Fried revealed a cynical view of ethics that seemed to contradict the more nuanced views of right and wrong he\'d discussed in the press before. "[M]an a lot of the dumb shit I said," he wrote. "[I]t\'s not true, not really."', 'https://www.vox.com/the-goods/23458837/sam-bankman-fried-ftx-sbf

In [40]:
references

["In December, Bankman-Fried was arrested in the Bahamas and charged with wire fraud, securities fraud, and money laundering, among other things; he has since been extradited to the US and released from jail on a $250 million bond -- according to Reuters, the largest-ever pretrial bond. A trial date has not been set, but it is expected to take place in the Southern District of New York. Caroline Ellison and Gary Wang, two former top executives at Bankman-Fried's companies, have pleaded guilty to",
 'https://www.vox.com/the-goods/23458837/sam-bankman-fried-ftx-sbf-downfall-explained#content',
 'In a Twitter DM interview with Future Perfect reporter Kelsey Piper following the implosion of FTX, Bankman-Fried revealed a cynical view of ethics that seemed to contradict the more nuanced views of right and wrong he\'d discussed in the press before. "[M]an a lot of the dumb shit I said," he wrote. "[I]t\'s not true, not really."',
 'https://www.vox.com/the-goods/23458837/sam-bankman-fried-ftx-

In [41]:
answer = get_answer(query, references)

answer

"Sam Bankman-Fried is an individual who has been involved in the financial industry and is the founder of FTX, a cryptocurrency exchange. He has recently faced legal charges and has been released on a $250 million bond. Bankman-Fried's former executives have also pleaded guilty to certain charges. (Vox)"

In [42]:
print(query)

Who is sam bankman fried


In [43]:
def get_answer_with_links(query, references):
    answers = []
    for i in range(0, len(references), 2):
        content = references[i]
        link = references[i + 1]
        answer = get_answer(query, [content])
        answers.append((answer, link))
    return answers

In [44]:
answers_with_links = get_answer_with_links(query, references)
for answer, link in answers_with_links:
    print("Answer:", answer)
    print("Link:", link)
    print()

Answer: Sam Bankman-Fried is an individual who was arrested in the Bahamas and charged with wire fraud, securities fraud, and money laundering, among other things. He has since been extradited to the US and released from jail on a $250 million bond, which is reported to be the largest-ever pretrial bond. Bankman-Fried is the founder of a cryptocurrency trading firm called FTX. (Reuters2021)
Link: https://www.vox.com/the-goods/23458837/sam-bankman-fried-ftx-sbf-downfall-explained#content

Answer: Sam Bankman-Fried is a prominent figure in the cryptocurrency industry. He is the founder and CEO of FTX, a cryptocurrency exchange platform. Bankman-Fried's views on ethics have been a subject of discussion, with some perceiving a contradiction between his previous statements and his recent comments in a Twitter DM interview (Piper2021). However, further analysis and additional sources are needed to provide a more comprehensive understanding of Bankman-Fried's background, accomplishments, and 